# Clase Nº 5

**Plan de la clase:**  
**(1)** Preguntas sobre la clase asincrónica. <br>
**(2)** Repaso de `pd.DataFrame.merge`. <br>
**(3)** Unir un dataframe con un geodataframe y visualizar atributos en un mapa <br>
**(4)** Unir dos geodataframes en base a atributos espaciales <br>
**(5)** Unir dos geodataframes en base a atributos espaciales <br>

### Preguntas sobre la clase asincrónica

### Repaso de `pd.DataFrame.merge`

In [ ]:
import pandas as pd

Creamos algunos DataFrame's de ejemplo

In [ ]:
# 
df1 = [["a", "b", "c", "d"], [1, 2, 3, 4]]
df1 = pd.DataFrame(df1).transpose()
df1

df2 = [["a", "b", "d"], [5, 6, 7]]
df2 = pd.DataFrame(df2).transpose()

# Como df3 pero le cambiamos el nombre a las columnas
df3 = [["a", "b", "c"], [8, 9, 10]]
df3 = pd.DataFrame(df3).transpose()

# Como df3 pero le cambiamos el nombre a las columnas
df4 = [["a", "b", "c"], [8, 9, 10]]
df4 = pd.DataFrame(df4).transpose()
df4.columns = ["a", "b"]

display(df1)
display(df2)
display(df3)
display(df4)

Si no especificamos nada, el tipo de merge es `inner` y las columnas utilizadas son todas aquellas con el mismo nombre, para ambos DataFrames. En este caso, `df1` y `df2`, son ambas columnas: `0` y `1`. En este ejemplo no hay valores compartidos en la columna `1` para esos dos DataFrames, por lo tanto nos la operación nos devuelve un DataFrame vacío.

In [ ]:
pd.merge(df1, df2)

Si no especificamos nombres de columnas y no hay columnas con el mismo nombre, (ej. `df1` y `df4`), nos da un error de tipo `MergeError`:

In [ ]:
try:
    pd.merge(df1, df4)
except pd.errors.MergeError:
    print("Estos DataFrame's no tiene columnas compartidas")

Especificamos que la unión sea en base a la columna `0`:

In [ ]:
pd.merge(df1, df2, on=0) # how="inner" por defecto

Podemos examinar distintos tipos de unión:

In [ ]:
display(pd.merge(df1, df2, on=0, how="inner"))
display(pd.merge(df1, df2, on=0, how="left"))
display(pd.merge(df2, df1, on=0, how="right"))
display(pd.merge(df2, df3, on=0, how="outer"))

Especificamos distintos sufijos para aquellas columnas que tienen el mismo nombre en los distintos DF's

In [ ]:
pd.merge(df1, df2, on=0, suffixes=("_1", "_2"), how="outer")

Usando los argumentos `left_on` y `right_on` para especificar distintas columnas de unión en cada DataFrame

In [ ]:
pd.merge(df2, df4, left_on=0, right_on="a", suffixes=("_1", "_2"), how="right")

#### `cross`
Este tipo de `merge` es diferente, lo podemos describir como "todos con todos". No se especifica en base a qué columnas unir, porque el DataFrame resultante es simplemente la combinación de todas las filas del primero con todas las filas del segundo.

In [ ]:
pd.merge(df2, df3, how='cross')

___

### Unir un geodataframe con un dataframe a través de un atributo no espacial

Vamos a ejemplificar cómo unir un `DataFrame` de Pandas con un `GeoDataFrame` de Geopandas.
Para eso vamos a utilizar una tabla que contiene la población de las distintas provincias argentinas, y el shapefile de las provincias que ya vimos en la clase asincrónica.

In [ ]:
import numpy as np
import pandas as pd
import geopandas as gpd

Descarguemos la tabla de población por provincia de [esta página](https://www.ign.gob.ar/NuestrasActividades/Geografia/DatosArgentina/Poblacion2) del sitio del IGN.
Para eso utilizamos la función `read_html` de `Pandas` que es capaz de extraer tablas de un archivo `html`.
Pueden necesitar instalar el paquete `lxml`, con `pip install lxml` en la terminal/cmd.

In [ ]:
url_pob = "https://www.ign.gob.ar/NuestrasActividades/Geografia/DatosArgentina/Poblacion2"
tablas = pd.read_html(url_pob)

In [ ]:
len(tablas)

In [ ]:
poblacion_df = tablas[2]

In [ ]:
poblacion_df.dtypes

Preprocesamos el dataframe de población:

In [ ]:
def convertir_a_int(fila, cols=["Año 2010", "Año 2015", "Año 2020"]):    

    '''
    Convertir cols, que contienen strings de números con "." como separador de miles, a tipo entero.
    '''
    
    try:
        for col in cols:
            fila[col] = int(fila[col].replace(".",""))
    except:
        pass
        
    return fila

In [ ]:
poblacion_df = poblacion_df.apply(convertir_a_int, axis=1)

In [ ]:
poblacion_df.dtypes

___

Ahora carguemos un GeoDataFrame con las provincias de Argentina:

In [ ]:
provincias_gdf = gpd.read_file(filename="datos_asincronica/provincia/provincia.zip")
provincias_gdf = provincias_gdf[["nam", "geometry"]]
provincias_gdf

Examinemos el mapa generado al plotear la geometría asociada a este `GeoDataFrame`.

In [ ]:
provincias_gdf.plot();

Bounding box para excluir a la Antártida:

In [ ]:
TdF_bounding_box = (-75,-60,-40,-55)
xmin, ymin, xmax, ymax = TdF_bounding_box

Usemos indexado de `GeoDataFrame`s (`gpd.GeoDataFrame.cx`) para seleccionar la fila correspondiente a Tierra del Fuego.

In [ ]:
TdF_Antartida_islas = provincias_gdf.cx[xmin:xmax, ymin:ymax]

¿Cómo luce el mapa para esta fila sola?

In [ ]:
provincias_gdf.cx[xmin:xmax, ymin:ymax].plot();

Notar que pese a que utilizamos una bounding box que excluía a la Antártida, el indexado por coordenadas no funcionó para excluirla. La razón es que esta jurisdicción es un multipolígono (`MultiPolygon`) y el indexado por coordenadas nos trae el multipolígono completo, no los polígonos individuales que pertenecen a la bounding box.

In [ ]:
from shapely.geometry.multipolygon import MultiPolygon

In [ ]:
TdF_bounding_box = (-75,-60,-40,-55)
xmin, ymin, xmax, ymax = TdF_bounding_box

TdF_muchos_poligonos = TdF_Antartida_islas.geometry.explode(index_parts=True)
TdF_muchos_poligonos = TdF_muchos_poligonos.cx[xmin:xmax, ymin:ymax]
TdF_muchos_poligonos = TdF_muchos_poligonos.to_list()

In [ ]:
TdF_multipol = MultiPolygon(TdF_muchos_poligonos)
TdF_multipol

___

### Ejercicio 1 (entre todos)

Crear una _bounding box_ similar a la creada para Tierra del Fuego, pero ahora para las Islas Malvinas. Seguir un procedimiento similar al realizado para Tierra del Fuego, finalmente agregar la geometría de las Islas Malvinas al multipolígono de Tierra del Fuego creado anteriormente. 

Modificar el Geodataframe original reemplazando el original por este.

### Sistemas de coordenadas de referencia

In [ ]:
provincias_gdf.crs

Los CRS **geográficos** no permiten calcular el área en kilómetros cuadrados. Necesitamos pasar a un CRS **proyectado**.

In [ ]:
provincias_gdf.area

In [ ]:
def auto_utm_zone(gdf):
    """Determinar la zona UTM y el hemisferio a partir de los valores de la bounding box."""
    
    # Calcular el centro de la bounding box
    centroid_lon = gdf.geometry.total_bounds[[0, 2]].mean()
    centroid_lat = gdf.geometry.total_bounds[[1, 3]].mean()
    
    # Determinar la zona UTM a partir de la longitud
    zone = int((centroid_lon + 180) // 6) + 1
    
    # Determinar el hemisferio (norte o sur) a partir del signo de la latitud
    hemisphere = 'north' if centroid_lat >= 0 else 'south'
    
    # El código EPSG para UTM depende de la zona, de la siguiente manera:
    epsg_code = f"EPSG:{32600 + zone if hemisphere == 'north' else 32700 + zone}"
    return epsg_code


utm_crs = auto_utm_zone(provincias_gdf)

# Ahora sí podemos calcular el área
provincias_gdf.to_crs(utm_crs).area / 1e6

### Ejercicio 2

Unir las bases `provincias_gdf` y `poblacion_df` por el nombre de la provincia. Notar que necesitamos especificar `left_on` y `right_on` porque los nombres de las columnas correspondientes son diferentes (o bien cambiar el nombre de una de las columnas).

- Examinar el tipo de objeto que resulta del `merge` usando `provincias_gdf` a la izquierda y `poblacion_df` a la derecha.
- Ídem anterior, con `poblacion_df` a la izquierda y `provincias_gdf` a la derecha.

A continuación, representar un mapa del país coloreando cada provincia en base a la variación relativa en su población en el período 2015-2020. Para el merge, usar el orden de (Geo)DataFrame's que corresponda.

### Ejercicio 3

Repetir el mapa anterior pero ahora colorear por densidad de población. Para eso, calcular previamente el área de cada provincia usando Geopandas. Recordar que necesitamos pasar a un CRS "proyectado" como hicimos arriba. Quitar a Capital Federal para que no desplace la escala demasiado. 

(_Ayuda_: cuando pasamos al CRS UTM, las coordenadas nos quedan en metros. Por lo tanto, como hicimos arriba, al calcular el área hay que dividir por 1.000.000)

___

### Unir geodatraframes por un atributo espacial: `sjoin`

**Ejercicio:** Leer el shapefile de departamentos como GeoDataFrame. Crear un widget reactivo que permita elegir una provincia y muestre un mapa con la división departamental de la misma.

Geopandas permite unir dos GeoDataFrames en base a relaciones espaciales entre las observaciones (filas) de cada uno.

Vamos a usar estas relaciones espaciales para poder vincular departamentos con provincias.

In [ ]:
departamentos_gdf = gpd.read_file("datos/departamento.zip")

# Extraemos las columnas de interés
# objectid es un identificador único para cada departamento a nivel país 
# Lo necesitamos porque hay departamentos cuyos nombres se repiten en distintas provincias.
departamentos_gdf = departamentos_gdf[["objectid", "nam", "geometry"]]

# Renombramos la columna que da el nombre del departamento
departamentos_gdf = departamentos_gdf.rename({"nam": "departamento"}, axis=1)

Examinemos este GeoDataFrame. ¿Qué problema nos encontramos?

In [ ]:
departamentos_gdf

El problema es que **no tenemos la provincia** asociada a cada departamento. 

Claro que podríamos conseguir esta relación de alguna forma pero, en lugar de eso, vamos a inferirla mediante **relaciones espaciales**.

Vamos a hacer lo siguiente: dado que conocemos los polígonos correspondientes a las provincias y a los departamentos, podemos preguntarnos _cuáles departamentos están dentro de qué provincias_.

El plan, entonces, será:
- obtener un punto que esté **dentro** de cada departamento del país, guardando el ID del departamento correspondiente. El nombre del departamento no sirve a este fin pues los nombres de departamento se repiten en distintas provincias.
    - Una opción sería usar el centroide (promedio de las coordenadas de los puntos), sin embargo esto puede fallar (¿en qué casos?).
    - En lugar de eso vamos a usar un método llamado `.representative_point()`.
- determinar qué puntos están ubicados dentro de qué (multi)polígonos de provincias. Esto nos daría la relación departamento -> provincia.

___

Primero, obtenemos los centroides de cada departamento.
Esto nos va a dar un `UserWarning`. ¿Por qué? Porque queremos calcular el centroide en un CRS geográfico: vamos a estar promediando latitudes y longitudes en lugar de distancias. Esto no nos preocupa, pues no estamos interesados en el valor exacto del centroide y, por otro lado, los departamentos son lo suficientemente chicos como para que esta diferencia no importe.

In [ ]:
departamentos_punto_adentro_gdf = departamentos_gdf.copy()
departamentos_punto_adentro_gdf["geometry"] = departamentos_gdf.geometry.representative_point();

Ahora realizamos el **join espacial** con `sjoin`. Para hacer un sjoin necesitamos un **predicado**, la relación espacial según la cual queremos unir.

Utilizamos el predicado `within`. Este es el tipo de relación que deseamos: queremos que el centroide de cada departamento esté adentro de (_within_) la geometría de la provincia.
Para ver una lista extensiva de predicados posibles se puede consultar [esta página de documentación de Shapely](https://shapely.readthedocs.io/en/stable/manual.html) (sección **Relationships**) o [este PDF](https://giswiki.hsr.ch/images/3/3d/9dem_springer.pdf). Otros valores posibles son `intersects`, `contains`, `overlaps` y `touches`.

In [ ]:
prov_dep_gdf = departamentos_centroid_gdf.sjoin(provincias_gdf, predicate="within")

In [ ]:
prov_dep_df = prov_dep_gdf[["objectid", "nam"]]

In [ ]:
prov_dep_df

In [ ]:
departamentos_gdf = departamentos_gdf.merge(prov_dep_df, how="inner", on="objectid")

In [ ]:
provincias = set(departamentos_gdf.nam.to_list())

In [ ]:
departamentos_gdf

Ahora representamos la división departamental de cada provincia en un mapa.

In [ ]:
import ipywidgets as widgets 
from ipywidgets import interact
from matplotlib import pyplot as plt

provincias_w = widgets.Dropdown(options=sorted(list(provincias)))  # ordenamos alfabéticamente

@interact
def mostrar_deptos_por_provincia(provincia=provincias_w):

    provincia_gdf = departamentos_gdf.query("nam == @provincia")
    
    provincia_gdf.plot(cmap="tab10", figsize=(10,10), edgecolor='black')
    
    # Añadir etiquetas
    for idx, row in provincia_gdf.iterrows():        
        depto = "\n".join(row['departamento'].split(maxsplit=1)
        plt.annotate(
            text=depto, 
            xy=(row['geometry'].centroid.x, row['geometry'].centroid.y),
            horizontalalignment='center', size=7
        )